In [3]:
# split
from sklearn.model_selection import train_test_split, StratifiedKFold
import os, shutil, random
import pandas as pd
import numpy as np

# shot_num = [1,2,4,8,16,32,64]
shot_num = "full_train"
exp_rp = "/home/r10user13/TOP/experiment"
exp_name = "lung_typing_full_train"
fold_num = 5

label_fp = "/home/r10user13/Capstone/TOP/experiment/LUNG/label.csv"

label_df = pd.read_csv(label_fp)
data_fp_label_map = dict(zip(
    list(label_df["wsi_fp"]), list(label_df["label"])
))

exp_rp_ = os.path.join(exp_rp, exp_name, 'input/csv/split')
if not os.path.exists(exp_rp_):
    os.makedirs(exp_rp_)
    
file_path = [i.replace("pkl", 'svs') for i in os.listdir("/data1/r10user13/TOP/lung_PLIP_20x_448/")]

luad_data = list(pd.read_csv("/home/r10user13/Capstone/TOP/experiment/LUAD/csv/bm.csv")["slide_path"])
luad_data = [j for j in luad_data if j.split("/")[-1] in file_path]
luad_label = ["luad"]*len(luad_data)

lusc_data = list(pd.read_csv("/home/r10user13/Capstone/TOP/experiment/LUSC/csv/bm.csv")["slide_path"])
lusc_data = [j for j in lusc_data if j.split("/")[-1] in file_path]
lusc_label = ["lusc"]*len(lusc_data)

# luad_train_data = [i for i in luad_train_data if i in file_list]
# lusc_train_data = [i for i in lusc_train_data if i in file_list]

if shot_num == "full_train":
    X = np.array(luad_data+lusc_data)
    y = np.array(luad_label+lusc_label)
    
    skf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=42)
    i = 0 
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = list(X[train_index]), list(X[test_index])
        y_train, y_test = list(y[train_index]), list(y[test_index])
        X_val, y_val = X_test, y_test
        out = pd.DataFrame({
            "data_path": X_train+X_test+X_val, 
            "label": y_train+y_test+y_val, 
            "type": ["train"]*len(X_train)+["test"]*len(X_test)+["val"]*len(X_val)
        })
        
        out.to_csv(os.path.join(exp_rp_, f"fold{i}.csv"))
        
        i+=1
        
else:
    
    luad_train_data, luad_test_data, luad_train_labels, luad_test_labels = train_test_split(
    luad_data, luad_label, test_size=0.2, stratify=luad_label, random_state=2023)

    lusc_train_data, lusc_test_data, lusc_train_labels, lusc_test_labels = train_test_split(
    lusc_data, lusc_label, test_size=0.2, stratify=lusc_label, random_state=2023)
    
    random.shuffle(luad_train_data)
    random.shuffle(lusc_train_data)
    
    for shot in shot_num:
        for i in range(fold_num):
            cur_i = i*shot
            luad_train_data_i = luad_train_data[cur_i:cur_i+shot]
            lusc_train_data_i = lusc_train_data[cur_i:cur_i+shot]
            
            luad_val_data_i = random.sample(list(set(luad_train_data)-set(luad_train_data_i)), len(luad_test_data))
            lusc_val_data_i = random.sample(list(set(lusc_train_data)-set(lusc_train_data_i)), len(lusc_test_data))
        
            data_path_list = luad_train_data_i+lusc_train_data_i+luad_test_data+lusc_test_data+luad_val_data_i+lusc_val_data_i
            data_label_list = [data_fp_label_map[data_path] for data_path in data_path_list]
        
            out = pd.DataFrame({
                "data_path": data_path_list, 
                "label": data_label_list, 
                "type": ["train"]*2*shot_num+["test"]*len(luad_test_data+lusc_test_data)+["val"]*len(luad_val_data_i+lusc_val_data_i)
            })
            out.to_csv(os.path.join(exp_rp_, f"fold{i}.csv"))